## 실제 활용해보며 익히는 MySQL 기본 + (중급) 문법
 - 다양한 문법을 나열하기보다는 실제 예제를 통해 필요할 때마다 문법을 익혀야 이해가 빠르다.

### 1. Schema 정의
 - TABLE 분리, FOREIGN KEY, PRIMARY KEY 사용

In [ ]:
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);

In [ ]:
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    dis_percent INT NOT NULL,
    provider VARCHAR(100)
);

##### 참고: 한글 처리에 문제가 있을 경우, DB, TABLE에 DEFAULT CHARSET=utf8 COLLATE=utf8_bin 옵션을 모두 추가해서 사용

```sql
CREATE DATABASE bestproducts DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
CREATE TABLE tablename(field definitions) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
```

In [ ]:
import pymysql 
db = pymysql.connect(host="localhost", port=3306, user="root", passwd="ck3270584!@#", db="bestproducts", charset="utf8")
cursor = db.cursor()

sql = '''
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    dis_percent INT NOT NULL,
    provider VARCHAR(100)
);
'''

cursor.execute(sql)

sql = '''
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
'''

cursor.execute(sql)

db.commit()
db.close()

### 2. 크롤링

#### main category 정보 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://corners.gmarket.co.kr/Bestsellers"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category("http://corners.gmarket.co.kr/"+category["href"], category.get_text())

#### main/sub category 정보 가져오기

### main/sub category + 상품 정보 + 상품 코드 + 판매자(제공자) 크롤링

In [5]:
import requests
from bs4 import BeautifulSoup

url = "http://corners.gmarket.co.kr/Bestsellers"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category("http://corners.gmarket.co.kr/"+category["href"], category.get_text())

KeyboardInterrupt: 

In [1]:
def get_category(category_link, category_name):
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, 'ALL')
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text())

In [2]:
def get_items(html, category_name, sub_category_name):
    items_result_list = list()
    best_item = html.select('div.best-list')
    for index, item in enumerate(best_item[1].select('li')):
        data_dict = dict()
        
        ranking = index + 1
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price em')
        
        if ori_price == None or ori_price.get_text() == '':
            ori_price = dis_price
            
        if dis_price == None:
            ori_price, dis_price = 0, 0
        else:
            ori_price = ori_price.get_text().replace(',', '').replace('원','')
            dis_price = dis_price.get_text().replace(',', '').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        else:
            dis_percent = dis_percent.get_text().replace('%', '')
            
        product_link = item.select_one('div.thumb > a')
        item_code = product_link.attrs['href'].split('=')[1].replace('&ver', '')
        
        res = requests.get(product_link.attrs['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a')
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
        
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['dis_percent'] = dis_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider
        
        save_data(data_dict)
#         print(category_name, sub_category_name, ranking, item_code, provider, title.get_text(), ori_price, dis_price, dis_percent)

### INSERT SQL 만들기

##### Table 구조
```sql
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
```

In [ ]:
item_info = {'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[머지포인트](머지포인트) 머지포인트 10만원권', 'ori_price': '100000', 'dis_price': '80000', 'dis_percent': '20', 'item_code': '2166726014', 'provider': '한국페이즈서비스'}

In [ ]:
sql = """
    INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('
    """ + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """',
    '""" + str(item_info['ranking']) + """',
    '""" + item_info['item_code'] + """')"""

In [ ]:
sql.replace('\n', '').strip().replace('    ', '')

### 목적
 - 실제로 데이터베이스를 사용하는 방법을 현업스타일로 설명을 드리는 것
 - SQL 강의 - 단순히 세세한 SQL 문법을 다 소개받고 끝 
 - 현실 세계의 데이터를 어떻게 테이블로 정의할 것인가
 - 어떻게 데이터를 넣을 것인가? (프로그래밍으로 넣는 것이 일반적)
 - 저장된 데이터를 분석한다. (SQL만 쓰는 경우 + 프로그래밍에서 데이터를 분석해서 풀스택으로 서비스화함)

##### Table 구조 
``` sql
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    dis_percent INT NOT NULL,
    provider VARCHAR(100)
);
```

In [ ]:
sql = """
    INSERT INTO items VALUES(
    '""" + item_info['item_code'] + """',
    '""" + item_info['title'] + """',
    """ + item_info['ori_price'] + """,
    """ + item_info['dis_price'] + """,
    '""" + item_info['dis_percent'] + """',
    '""" + item_info['provider'] + """'
)"""

In [ ]:
sql.replace('\n','').strip().replace('    ', '')

In [3]:
def save_data(item_info):
    import pymysql 
    db = pymysql.connect(host="localhost", port=3306, user="root", passwd="ck3270584!@#", db="bestproducts", charset="utf8")
    cursor = db.cursor()
    
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql)
    result = cursor.fetchone()
    if result[0] == 0:
        sql = """
        INSERT INTO items VALUES(
        '""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """',
        """ + str(item_info['ori_price']) + """,
        """ + str(item_info['dis_price']) + """,
        """ + str(item_info['dis_percent']) + """,
        '""" + item_info['provider'] + """')"""
        sql.replace('\n','').strip().replace('    ', '')
        cursor.execute(sql)
        
    sql = """
    INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES('
    """ + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """',
    '""" + str(item_info['ranking']) + """',
    '""" + item_info['item_code'] + """')"""
    sql.replace('\n','').strip().replace('    ', '')
    cursor.execute(sql)

#### COUNT SQL
 - COUNT: 검색 결과의 row 수를 가져올 수 있는 SQL 문법
 - SQL 예제: SELECT COUNT(*) FROM items WEHRE item_code = '111110000'

```sql 
sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
cursor.execute(sql)
result = cursor.fetchone()
print(result[0])
```

In [ ]:
sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""

In [ ]:
sql